In [ ]:
#https://www.kdnuggets.com/2020/12/implementing-adaboost-algorithm-from-scratch.html
#https://github.com/jinxin0924/multi-adaboost/blob/master/multi_AdaBoost.py

In [3]:
from sklearn.datasets import make_gaussian_quantiles
import numpy as np

from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import AdaBoostClassifier

In [4]:
X, y = make_gaussian_quantiles(
    n_samples=13000, n_features=10, n_classes=3, random_state=1
)
y = np.where(y==0,-1,1)

n_split = 3000

X_train, X_test = X[:n_split], X[n_split:]
y_train, y_test = y[:n_split], y[n_split:]


In [ ]:
#------------------------------- BINARY CLASSIFICATIONS ---------------------- #

class BinaryClassAdaboost():
    """
    """
    
    def __init__(self, n_estimators:int):
        """
        Initialialisation of Adaboost class
        Parameters: 
            n_estimators: int:  number of weak learners 
        """
        self.n_estimators = n_estimators
        self.list_WL = [] #list with model
        self.list_alpha = [] #list with weight of model 
        
        
        
    def fit(self, X, y):
        """
        Fit model 
        Parameters: 
            X: array: data
            y: array: vector of class labels where yi E Y= {1,..., k} and k = 2
        """
        ## Step 1: Initialize the weights to a constant
        n_samples = X.shape[0]                
        w = []
        ##Weights are initialized to 1/Number of samples: 
        w_t = [1/n_samples for x in range(n_samples)]       
              
        
        ## Step 2: Classify with ramdom sampling of data using a weak learner
        #Construction des weaklearner
        
        #for each weak learner
        for t in range(self.n_estimators):

            #Choose and Call the Base/Weak learner
            #A decision tree with one depth has one node and is called a stump or weak learner
            WL = DecisionTreeClassifier(max_depth=1,)
            #Fit the stump model with the ramdom samples
            WL.fit(X, y, sample_weight=w_t)
            #Get the predicted classes
            y_pred = WL.predict(X)
            
            ##Step 3: Compute error of weak learner
            eps = self.error_wl(w_t, y_pred, y)
        
            # if the error of the weak learner is higher then 0.5 (worse then random guess) 
            #don't take into account this learner weight
            if eps > 0.5:
                break
            
            #Step 4: Calculate the performance of the weak learner
            #Performance of the weak learner(α) = 0.5* ln (1 – error/error)
            #Calculate alpha for this weak learner
            alpha_t = 0.5 * log((1- eps) / eps)
            
            
            #Step 5: Update weight
            #With the alpha performance (α) the weights of the wrongly classified records are increased
            #and the weights of the correctly classified records decreased.
            y_temp = np.multiply(y, y_pred)
            y_temp2 = -alpha_t * y_temp 
            w_t = np.multiply(w_t, np.exp(y_temp2))

            #normalizing the weigths for the sum to be equal do 1
            w_t = w_t / sum(w_t)
            
            #store the alpha performance of each weak learner
            self.list_alpha.append(alpha_t)
            #store each weak learner
            self.list_WL.append(WL)
            
            
            
        return 1

    def predict(self, X):
        """
        predict output of Adaboost 
        Paramters: 
            X: array: data
        Return: 
            y_pred: array: data
        """
        #The final prediction is a compromise between all the weak learners predictions
        list_y_pred = []
        
        #for each weak learner get their prediction
        for WL in self.list_WL:
            list_y_pred.append(WL.predict(X))
         
        #the array of all the predictions
        arr_y_pred = np.array(list_y_pred)
        
        #Final prediction is obtained by the weighted by alpha sum of each weak learner prediction
        weighted_sum_pred = np.sum(np.multiply(arr_y_pred, self.list_alpha), axis = 0)
        y_pred_resized = np.reshape(weighted_sum_pred, (weighted_sum_pred.shape[0],1))
        
        #get -1 if y_pred < 0 or 1 if y_pred > 0
        y_pred = np.sign(y_pred_resized)
        
        return y_pred 
        
    def error_wl(self, w_t, y_pred, y):
        """
        error of current weaklearner
        Parameters:
            w_t: array:  weight of observation
            y_pred: array: output of wl 
            y: array: labels
        Return: 
            eps: float: error of wl 
        """
        
        ind_err = []
        for i in range(y_pred.shape[0]):
            if y_pred[i] != y[i]:
                ind_err.append(1) 
            else: 
                ind_err.append(0) 
    
        w_ind_err = np.multiply(w_t,ind_err)
        
        eps = np.sum(w_ind_err)
    
        return eps
    
        

In [ ]:
#------------------------------- BINARY CLASSIFICATIONS WITH SAMPLING ---------------------- #

class BinaryClassAdaboost():
    """
    """
    
    def __init__(self, n_estimators:int):
        """
        Initialialisation of Adaboost class
        Parameters: 
            n_estimators: int:  number of weak learners 
        """
        self.n_estimators = n_estimators
        self.list_WL = [] #list with model
        self.list_alpha = [] #list with weight of model 
        
        
        
    def fit(self, X, y):
        """
        Fit model 
        Parameters: 
            X: array: data
            y: array: vector of class labels where yi E Y= {1,..., k} and k = 2
        """
        ## Step 1: Initialize the weights to a constant
        n_samples = X.shape[0]                
        w = []
        ##Weights are initialized to 1/Number of samples: 
        w_t = [1/n_samples for x in range(n_samples)]       
              
        
        ## Step 2: Classify with ramdom sampling of data using a weak learner
        #Construction des weaklearner
        
        #for each weak learner
        for t in range(self.n_estimators):
            
            #Draw random samples with replacement from original data   
            #with the updated weigths
            X_sample, y_sample = self.sampling(X, y, w_t)
            
            #Choose and Call the Base/Weak learner
            #A decision tree with one depth has one node and is called a stump or weak learner
            WL = DecisionTreeClassifier(max_depth=1,)
            #Fit the stump model with the ramdom samples
            WL.fit(X_sample, y_sample, sample_weight=w_t)
            #Get the predicted classes
            y_pred = WL.predict(X)
            
            ##Step 3: Compute error of weak learner
            eps = self.error_wl(w_t, y_pred, y)
        
            # if the error of the weak learner is higher then 0.5 (worse then random guess) 
            #don't take into account this learner weight
            if eps > 0.5:
                break
            
            #Step 4: Calculate the performance of the weak learner
            #Performance of the weak learner(α) = 0.5* ln (1 – error/error)
            #Calculate alpha for this weak learner
            alpha_t = 0.5 * log((1- eps) / eps)
            
            
            #Step 5: Update weight
            #With the alpha performance (α) the weights of the wrongly classified records are increased
            #and the weights of the correctly classified records decreased.
            y_temp = np.multiply(y, y_pred)
            y_temp2 = -alpha_t * y_temp 
            w_t = np.multiply(w_t, np.exp(y_temp2))

            #normalizing the weigths for the sum to be equal do 1
            w_t = w_t / sum(w_t)
            
            #store the alpha performance of each weak learner
            self.list_alpha.append(alpha_t)
            #store each weak learner
            self.list_WL.append(WL)
            
            
            
        return 1

    def predict(self, X):
        """
        predict output of Adaboost 
        Paramters: 
            X: array: data
        Return: 
            y_pred: array: data
        """
        #The final prediction is a compromise between all the weak learners predictions
        list_y_pred = []
        
        #for each weak learner get their prediction
        for WL in self.list_WL:
            list_y_pred.append(WL.predict(X))
         
        #the array of all the predictions
        arr_y_pred = np.array(list_y_pred)
        
        #Final prediction is obtained by the weighted by alpha sum of each weak learner prediction
        weighted_sum_pred = np.sum(np.multiply(arr_y_pred, self.list_alpha), axis = 0)
        y_pred_resized = np.reshape(weighted_sum_pred, (weighted_sum_pred.shape[0],1))
        
        #get -1 if y_pred < 0 or 1 if y_pred > 0
        y_pred = np.sign(y_pred_resized)
        
        return y_pred 
        
    def error_wl(self, w_t, y_pred, y):
        """
        error of current weaklearner
        Parameters:
            w_t: array:  weight of observation
            y_pred: array: output of wl 
            y: array: labels
        Return: 
            eps: float: error of wl 
        """
        
        ind_err = []
        for i in range(y_pred.shape[0]):
            if y_pred[i] != y[i]:
                ind_err.append(1) 
            else: 
                ind_err.append(0) 
    
        w_ind_err = np.multiply(w_t,ind_err)
        
        eps = np.sum(w_ind_err)
    
        return eps
    
        
    def sampling(self, X, y, w_t):
        """
        sampling X with w_t 
        Parameters:
            X: array: data
            y: array: labels
            w_t: array: weigth
        Return:
            X_sample: array: sample of X
            y_sample: array: labels corresponding to X_sample
        """
        
        '''As for our implementaion of AdaBoost 
        y needs to be in {-1,1}
        '''
        y = np.where(y==0,-1,1)
        
        #put X and y in same array to sample 
        y_temp = np.reshape(y, (y.shape[0], 1))
        
        data = np.hstack((X, y_temp))
    
        #size of sample
        size = int(0.75*X.shape[0])
        
        #get sampled data with the weights 
        sample = random.choices(data, weights=w_t, k=size)
        
        #getting X and y 
        y_sample = sample[:,-1]
        X_sample = sample[:,:-1]
        
        return X_sample, y_sample

In [41]:
#-------------------------------Multiclass LASSIFICATIONS ---------------------- #


class MultiClassAdaBoost(object):
    '''
    Parameters
    -----------
    base_estimator: object
        The base model from which the boosted ensemble is built.
    n_estimators: integer, optional(default=50)
        The maximum number of estimators
    learning_rate: float, optional(default=1)
    Attributes
    -------------
    estimators_: list of base estimators
    estimator_weights_: array of floats
        Weights for each base_estimator
    estimator_errors_: array of floats
        Classification error for each estimator in the boosted ensemble.
    '''

    def __init__(self, n_estimators, learning_rate):
        self.n_estimators = n_estimators
        self.list_WL = [] #list with model
        self.list_alpha = [] #list with weight of model 
        self.learning_rate_ = learning_rate
        self.estimator_errors = []


    def fit(self, X, y):
        
        ## Step 1: Initialize the weights to a constant
        n_samples = X.shape[0]                
        w = []
        ##Weights are initialized to 1/Number of samples: 
        w_t = [1/n_samples for x in range(n_samples)]       
        
        # So in boost we have to ensure that the predict results have the same classes sort
        self.classes_ = np.array(sorted(list(set(y))))
        self.n_classes_ = len(self.classes_)
        
        
        ## Step 2: Classify with ramdom sampling of data using a weak learner
        #Construction des weaklearner
        
        #for each weak learner
        for t in range(self.n_estimators):
          
            #Choose and Call the Base/Weak learner
            #A decision tree with one depth has one node and is called a stump or weak learner
            WL = DecisionTreeClassifier(max_depth=1)
            #Fit the stump model with the ramdom samples
            WL.fit(X, y, sample_weight=w_t)
            
            y_pred = WL.predict(X)
            
            ##Step 3: Compute error of weak learner
            incorrect = y_pred != y
            estimator_error = np.dot(incorrect, w_t) / np.sum(w_t, axis=0)
            
            # if worse than random guess, stop boosting
            if estimator_error >= 1 - 1 / self.n_classes_:
                break

            # update alphe performance
            alpha_t = self.learning_rate_ * np.log((1 - estimator_error) / estimator_error) + np.log(
            self.n_classes_ - 1)
        

            # update sample weight
            w_t *= np.exp(alpha_t * incorrect)
            sample_weight_sum = np.sum(w_t, axis=0)

            # normalize sample weight
            w_t /= sample_weight_sum
            
            #store the alpha performance of each weak learner
            self.list_alpha.append(alpha_t)
            #store each weak learner
            self.list_WL.append(WL)
            # append error
            self.estimator_errors.append(estimator_error)

        return self


    def predict(self, X):
        n_classes = self.n_classes_
        classes = self.classes_[:, np.newaxis]

        
        pred = sum((estimator.predict(X) == classes).T * w
                   for estimator, w in zip(self.list_WL,
                                           self.list_alpha))

        pred /= sum(self.list_alpha)
        if n_classes == 2:
            pred[:, 0] *= -1
            pred = pred.sum(axis=1)
            return self.classes_.take(pred > 0, axis=0)

        return self.classes_.take(np.argmax(pred, axis=1), axis=0)

In [42]:
model = MultiClassAdaBoost(3, 0.01)

In [43]:
model.fit(X,y)

In [46]:
X_pred =  np.array([0.4, 0.3, 0.5, 0.6, 0.8, 0, 1, 0.9, 0.7,0.03])

In [48]:
model.predict(X_pred.reshape(1, -1))

array([2])

In [ ]:
#Multi Class

from sklearn import base

def indexToVector(y,k,labelDict):
    y_new = []
    for yi in y:
        i = labelDict[yi]
        v = np.ones(k)*(-1/(k-1))
        v[i] = 1
        y_new.append(v)
    return np.array(y_new)

def indexToLabel(i,clf):
    return clf.classes[i]

class AdaBoostClassifier:
    
    def __init__(self,base_estimator=None,n_estimators=50):
        self.n_estimators = n_estimators
        self.models = [None]*n_estimators
        if base_estimator == None:
            base_estimator = DecisionTreeClassifier(max_depth=1)
        self.base_estimator = base_estimator
        self.estimator_errors_ = []
        
    def fit(self,X,y):
        
        X = np.float64(X)
        N = len(y)
        w = np.array([1/N for i in range(N)])
        
        self.createLabelDict(np.unique(y))
        k = len(self.classes)
        
        for m in range(self.n_estimators):
            
            Gm = base.clone(self.base_estimator).\
                            fit(X,y,sample_weight=w).predict
            
            incorrect = Gm(X) != y
            errM = np.average(incorrect,weights=w,axis=0)
            
            self.estimator_errors_.append(errM)
            
            BetaM = (np.log((1-errM)/errM)+np.log(k-1))
            
            w *= np.exp(BetaM*incorrect*(w > 0))
            
            self.models[m] = (BetaM,Gm)
            
    def createLabelDict(self,classes):
        self.labelDict = {}
        self.classes = classes
        for i,cl in enumerate(classes):
            self.labelDict[cl] = i

    def predict(self,X):
        k = len(self.classes)
        y_pred = sum(Bm*indexToVector(Gm(X),k,self.labelDict) \
                             for Bm,Gm in self.models)
        
        iTL = np.vectorize(indexToLabel)
        return iTL(np.argmax(y_pred,axis=1),self)

In [ ]:
## Explore Weak Learner (use different weak learners or with different parameters)


In [ ]:
## Explore Number of Trees


In [ ]:
##Comparing with other algo

In [ ]:
#sklearn adaboost
